In [1]:
from kiteconnect import KiteConnect
from kiteconnect import KiteTicker
import secret

api_key = secret.api_key
api_secret = secret.api_secret

kite = KiteConnect(api_key=api_key)

In [2]:
print(kite.login_url())

https://kite.zerodha.com/connect/login?api_key=x5ugrx91yexerba1&v=3


In [3]:
data = kite.generate_session("5gP3QnSaiDE44SK64Sg7hGMP7oe0OZrO", api_secret=api_secret)
kite.set_access_token(data["access_token"])
access_token = data["access_token"]
kws = KiteTicker(api_key, access_token)
print(data["access_token"])

Aj1HPjcRZTAYWsdxLBtMhizOQBv3hUVY


In [23]:
def find_neighbour(sorted_data,compare_num):
    lower_neighbors = []
    higher_neighbors = []
    
    for num in sorted_data:
        if num < compare_num:
            lower_neighbors.append(num)
        elif num > compare_num:
            higher_neighbors.append(num)

    # Get first, second, and third lower neighbors (if available)
    first_lower = lower_neighbors[-1] if len(lower_neighbors) > 0 else None
    second_lower = lower_neighbors[-2] if len(lower_neighbors) > 1 else None
    third_lower = lower_neighbors[-3] if len(lower_neighbors) > 2 else None

    # Get first, second, and third higher neighbors (if available)
    first_higher = higher_neighbors[0] if len(higher_neighbors) > 0 else None
    second_higher = higher_neighbors[1] if len(higher_neighbors) > 1 else None
    third_higher = higher_neighbors[2] if len(higher_neighbors) > 2 else None

    return first_lower,second_lower,third_lower,first_higher,second_higher,third_higher


In [24]:
def find_low_neighbor(sorted_data,compare_num):
    neighbourLow = None
    for num in sorted_data:
        if num < compare_num:
            neighbourLow =  num
        else: break
    return neighbourLow

In [25]:
def percentage(close_price,percent_value):
    percnt = (close_price/100)*percent_value
    addprcntge = close_price+percnt
    subprcntge = close_price-percnt
    return addprcntge,subprcntge

In [26]:
def comparingLtp(stockLTP,neighbourList):
    stockList = []
    addpercentage,subpercentage = percentage(stockLTP,1)
    
    for num in neighbourList:
        
        try:
            if subpercentage <= num <= addpercentage:
                stockList.append(num)
        except: continue
    if stockList: return stockList

In [ ]:
import pandas as pd
import os
import json
from datetime import date

stockDict = dict()
stock =  pd.read_csv(f"watchList\\Midcap 100.csv")
leveltap = pd.read_csv(f"otherImp\\M\\Midcap 100.csv")
instruments = kite.instruments("NSE")
symbols = stock['Symbol']
count = 0 
for symbol in symbols:
    
    to_date = date.today()
    from_date = "2019-10-01"
    try:
        instrument_token = next(item["instrument_token"] for item in instruments if item["tradingsymbol"] == symbol)
        data = kite.historical_data(instrument_token,from_date,to_date,"day")
        df = pd.DataFrame(data)

        df['date'] = pd.to_datetime(df['date']).dt.to_period('M') # WE MADE CHANGES FORM 'M' TO 'W' WHICH CONVERTED INTO WEEKS FROM MONTHS
        monthly_data = df.groupby('date').agg({
            'open': 'first',
            'high': 'max',
            'low': 'min',
            'close': 'last',
            'volume': 'sum'
        }).reset_index()

        last_month = len(monthly_data)-2
        close_data = monthly_data['close'][last_month]
        level = leveltap['levels'][count]
        count+=1
        data = json.loads(level)
        sorted_data =  sorted(map(float,data.keys()))
       
        first_lower,second_lower,third_lower,first_higher,second_higher,third_higher = find_neighbour(sorted_data,close_data)   
        ltp = kite.ltp('NSE:'+symbol)
        stockLTP = ltp['NSE:'+symbol]['last_price']
        addprecentages,subpercentage = percentage(stockLTP,1)
        stockList = comparingLtp(stockLTP,[second_lower,third_lower,second_higher,third_higher])
        if stockList:
            stockDict[symbol] = [stockLTP, stockList, float(close_data),[first_lower,second_lower,third_lower,first_higher,second_higher,third_higher]]
            
            
    except: continue
import pprint
pprint.pprint(stockDict)
    








